In [1]:
import numpy as np
from jesse import helpers, research

from custom_indicators.all_features import feature_bundle

_, trading_1m = research.get_candles(
    "Binance Perpetual Futures",
    "BTC-USDT",
    "1m",
    helpers.date_to_timestamp("2021-01-01"),
    helpers.date_to_timestamp("2025-01-31"),
    warmup_candles_num=0,
    caching=False,
    is_for_jesse=False,
)

_, trading_3m = research.get_candles(
    "Binance Perpetual Futures",
    "BTC-USDT",
    "3m",
    helpers.date_to_timestamp("2021-01-01"),
    helpers.date_to_timestamp("2025-01-31"),
    warmup_candles_num=0,
    caching=False,
    is_for_jesse=False,
)

_, trading_15m = research.get_candles(
    "Binance Perpetual Futures",
    "BTC-USDT",
    "15m",
    helpers.date_to_timestamp("2021-01-01"),
    helpers.date_to_timestamp("2025-01-31"),
    warmup_candles_num=0,
    caching=False,
    is_for_jesse=False,
)

In [2]:
features_1m = feature_bundle(trading_1m, sequential=True)
features_1m = {f"1m_{k}": v[::3] for k, v in features_1m.items()}
features_3m = feature_bundle(trading_3m, sequential=True)
features_15m = feature_bundle(trading_15m, sequential=True)
features_15m = {f"15m_{k}": np.repeat(v, 5) for k, v in features_15m.items()}

len_3m = trading_3m.shape[0]
for k, v in features_1m.items():
    assert len(v) == len_3m, f"{k} has length {len(v)} not {len_3m}"
for k, v in features_3m.items():
    assert len(v) == len_3m, f"{k} has length {len(v)} not {len_3m}"
for k, v in features_15m.items():
    assert len(v) == len_3m, f"{k} has length {len(v)} not {len_3m}"

In [3]:
import pandas as pd

df_features = pd.DataFrame({**features_1m, **features_3m, **features_15m})
print(df_features.shape)
df_features.head()

(715680, 1263)


,1m_acc_swing_index,1m_acc_swing_index_lag1,1m_acc_swing_index_lag2,1m_acc_swing_index_lag3,1m_acc_swing_index_dt,1m_acc_swing_index_ddt,1m_ehlers_early_onset_trend,1m_ehlers_early_onset_trend_lag1,1m_ehlers_early_onset_trend_lag2,1m_ehlers_early_onset_trend_lag3,...,15m_swamicharts_stochastic_34,15m_swamicharts_stochastic_35,15m_swamicharts_stochastic_36,15m_swamicharts_stochastic_37,15m_swamicharts_stochastic_38,15m_swamicharts_stochastic_39,15m_swamicharts_stochastic_40,15m_swamicharts_stochastic_41,15m_swamicharts_stochastic_42,15m_swamicharts_stochastic_43
0,0.000000,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.028500,0.036568,0.048011,0.000000,-0.008068,0.003376,-1.0,-1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-0.011203,-0.023382,0.026734,0.028500,0.012179,0.062296,-1.0,-1.0,-1.0,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-0.121147,-0.070155,-0.020838,-0.011203,-0.050992,-0.001675,-1.0,-1.0,-1.0,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-0.129019,-0.160702,-0.119638,-0.121147,0.031683,0.072747,-1.0,-1.0,-1.0,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df_features.to_parquet("data/features_d002.parquet", index=False)